# Evaluation<a name = "Top"></a>

# Quick Links

<ol>
    <li><a href = #setup>Setup</a></li>
    <li><a href = #store>Save File</a></li>
</ol>

## Imports

In [1]:
import os
import shutil

In [2]:
import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import tensorflow as tf
  
available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from tensorflow.compat.v1.keras import callbacks, backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model

from datetime import datetime, timedelta
from dataclasses import dataclass
import time
import pandas as pd
import numpy as np
import shutil

from enum import Enum

from numpy import array
from numpy.random import seed
import random

import ntpath

import copy
import re

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

Using TensorFlow backend.


The installed version of TensorFlow 2.1.0 includes GPU support.

Num GPUs Available:  2 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2494487368757663439
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10058596352
locality {
  bus_id: 1
  links {
  }
}
incarnation: 403864703925656321
physical_device_desc: "device: 0, name: TITAN Xp COLLECTORS EDITION, pci bus id: 0000:3b:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10058596352
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14224681829264369836
physical_device_desc: "device: 1, name: TITAN Xp COLLECTORS EDITION, pci bus id: 0000:5e:00.0, compute capability: 6.1"
]


In [3]:
os.environ['PYTHONHASHSEED']='0'

s = 1
seed(s)
tf.random.set_seed(s)
random.seed(s)
np.random.seed(s)

## Deep Nets

### ALEX Net

In [4]:
def alexnet(activation, leaky_alpha, dropout):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential([
        Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation=activation_layer, input_shape=(224,224,Global.num_image_channels)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation=activation_layer, padding="same"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        Flatten(),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(4096, activation=activation_layer),
        Dropout(dropout),
        Dense(units = 2, activation=activation_layer)
        #Dense(10, activation='softmax')
    ])
    return model

### VGG16

In [5]:
def vgg16(activation, leaky_alpha):
        
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    
    model.add(Conv2D(input_shape=(224,224,Global.num_image_channels), filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = activation_layer))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(units = 4096, activation = activation_layer))
    model.add(Dense(units = 4096, activation = activation_layer))
    #model.add(Dense(units = 2, activation = "softmax"))
    #model.add(Dense(units = 2, activation=activation_layer))
    model.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))

    #opt = Adam(lr = 0.001)
    #model.compile(optimizer = opt, loss= keras.losses.categorical_crossentropy, metrics = ['accuracy'])
    model.summary()
    return model

### Pre-trained VGG16

In [6]:
def pre_vgg16(activation, leaky_alpha, dropout, hidden_layer, first_neuron):
    
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
    
    model = Sequential()
    cnn = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

    for layer in cnn.layers[:15]:
        layer.trainable = False
        #print(layer.name, layer.trainable)
    
    model.add(cnn)
    
    fc = Sequential()
    fc.add(Flatten(input_shape = model.output_shape[1:])) # (7, 7, 512) 

    fc.add(Dense(units = first_neuron , kernel_initializer = glorot_uniform(seed = 1)))
    fc.add(activation_layer)
    if dropout > 0.0:
        fc.add(Dropout(rate = dropout))

    hidden_neuron_fraction = first_neuron
    for i in range(hidden_layer):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        fc.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
        fc.add(activation_layer)
        if dropout > 0.0:
            fc.add(Dropout(rate = dropout))

    fc.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))
    
    model.add(fc)
    return model

## ResNet

In [7]:
def resnet(activation, leaky_alpha):
         
    activation_layer = ReLU()
    if activation == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = leaky_alpha)
    elif activation == 'relu':
        activation_layer = ReLU()
        
    model = Sequential()
    cnn = ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=(224, 224, Global.num_image_channels))
    for layer in cnn.layers:
        layer.trainable = True
    x = cnn.output

    x = AveragePooling2D((7, 7), padding='same')(x)
    x = Flatten()(x)

    x = Dense(units = 512, activation = activation_layer)(x)
    x = Dense(units = 256, activation = activation_layer)(x)
    x = Dense(units = 64, activation = activation_layer)(x)
    x = Dense(units = 2, activation = activation_layer)(x)
    model = Model(cnn.input, x)

    return model

## Hilfsfunktionen

### Benutzerdefinierte Kostenfunktion & Metrik

In [8]:
def mean_absolut_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Datenpipeline für Test

In [9]:
def setup_data():
    
    if Global.mode == 'SYNTH':
        df = pd.read_csv(Global.csv_file)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_test = df_shuffled[df_shuffled.shape[0] - 10000 : df_shuffled.shape[0]]

    elif Global.mode == 'REAL':
        df_real = pd.read_csv(Global.csv_file)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_test_real = df_shuffled_real[df_shuffled_real.shape[0] - 862 : df_shuffled_real.shape[0]]
        df_test = df_test_real
        
    #elif Global.mode == TrainingSet.MIXED:
    #    print('Inferenz auf gemischten Testdaten nicht unbedingt sinnvoll')
    #    df = pd.read_csv(_CSV_FILE)
    #    df_shuffled = df.sample(frac = 1, random_state = 1)
    #    df_test = df_shuffled[df_shuffled.shape[0] - 10000 : df_shuffled.shape[0]]
    #    
    #    df_real = pd.read_csv(_CSV_FILE_REAL)
    #    df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
    #    df_test_real = df_shuffled_real[df_shuffled_real.shape[0] - 61 : df_shuffled_real.shape[0]]
    #    df_test = df_test.drop(df_test.index[0 : 61])
    #    df_test = df_test.append(df_test_real)  
    #
    
    test_data_generator = ImageDataGenerator(
        rescale = 1./255
    )

    test_generator = test_data_generator.flow_from_dataframe(
        dataframe = df_test,
        directory = Global.image_dir,
        x_col = 'Filename',
        y_col = ['Elevation', 'Azimuth'],
        color_mode= Global.image_channels,
        class_mode = 'raw',
        target_size = (224, 224),
        seed=1,
        shuffle = False,
        batch_size = 1
    )
    
    return test_generator, df_test

### Trainingsset-Typ nach String Converter

### Modell für Inferenz

In [10]:
def model_predict(parameters, prediction_runs = 1):
    
    K.clear_session()
    
    test_generator, df_test = setup_data()

    if(Global.net_architecture == 'ALEX'):
        model = alexnet(parameters['activation'], parameters['leaky_alpha'], parameters['dropout_rate'])
    elif(Global.net_architecture == 'VGG16'):
        model = vgg16(parameters['activation'], parameters['leaky_alpha'])
    elif(Global.net_architecture == 'preVGG16'):
        if(Global.image_channels == 'rgba'):
            print('Error: Please use rgb as input channel!!!!!!')
            return null 
        model = pre_vgg16(parameters['activation'], parameters['leaky_alpha'], parameters['dropout_rate'], parameters['hidden_layers'], parameters['first_neuron'])
    elif(Global.net_architecture == 'RESNET'):
        model = resnet(parameters['leaky_alpha'], parameters['leaky_alpha'])
    else:
        print('Wrong net architecture!')
        
    model.load_weights(parameters['model_to_load'])
    ##print('Using Optimizer: {} with Learning Rate: {}'.format(parameters['optimizer'][0], parameters['learning_rate']))
    #model.compile(
    #    optimizer = parameters['optimizer'][0](lr = parameters['learning_rate']),
    #    loss = Global.loss_function, 
    #    metrics = get_Reduction_Metric(Global.reduction_metric)
    #)
    model.compile(
        optimizer = parameters['optimizer'](lr = lr_normalizer(parameters['learning_rate'], parameters['optimizer'])), 
        loss = Global.loss_function, 
        metrics = get_reduction_metric(Global.reduction_metric)
    )

    test_generator.reset()

    #print("Predicting using these values:\nTest Data: {}\nUsing Loss: {} on Dataset: {}".format(, parameters['dataset_name']))
    print('Net: {}'.format(Global.net_architecture))
    print('Image Channels: {}'.format(Global.image_channels))
    duration = 0    
    
    print('test_generator: {}'.format(test_generator))
    steps = test_generator.n // test_generator.batch_size
    
    print('test_generator.n: {}'.format(test_generator.n))
    print('test_generator.batch_size: {}'.format(test_generator.batch_size))
    print('steps: {}'.format(steps))
    for n in range(prediction_runs):
        startTime = datetime.now()
        #predictions = model.predict_generator(generator = test_generator, steps = test_generator.n // test_generator.batch_size, verbose = 0)
        predictions = model.predict(x = test_generator, steps = test_generator.n // test_generator.batch_size, verbose = 0)
        duration = (datetime.now() - startTime).total_seconds()
        time.sleep(1)
    
    time_per_prediction = duration / prediction_runs
    time_per_image = time_per_prediction / len(predictions)
    print("Prediction repeated {} times at a total time of {}sec. \nAverage Time per Prediction: {} sec. Average Time per Image: {} sec".format(prediction_runs, duration, time_per_prediction, time_per_image))

    del model
    return predictions, df_test, duration, time_per_image

### Load Params

In [11]:
def load_params(dataframe, model_to_load):
    params = copy.deepcopy(p)
    params['model_to_load'] = model_to_load
    params['optimizer'] = make_optimizer("<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>")
    params['learning_rate'] = dataframe.iloc[0]['lr']
    params['first_neuron'] = dataframe.iloc[0]['first_neuron']
    params['dropout_rate'] = dataframe.iloc[0]['dropout']
    params['activation'] = dataframe.iloc[0]['activation']
    params['leaky_alpha'] = dataframe.iloc[0]['leaky_alpha']
    params['hidden_layers'] = dataframe.iloc[0]['hidden_layers']
    # -------------------------------------------------------------
    return params

## Hilfsfunktionen

### Radians $\rightarrow$ Degree

In [12]:
def to_degree(angle_in_rad):
    return angle_in_rad * 180 / np.pi

### Degree $\rightarrow$ Radians

In [13]:
def to_radians(angle_in_deg):
    return angle_in_deg * np.pi / 180

### Sphärische $\rightarrow$ Karthesische Koordinaten

In [14]:
def spheric_cartesian_polar(phi_d, theta_d):
    x = np.sin(np.radians(90.0 - theta_d)) * np.cos(np.radians(phi_d))
    y = np.sin(np.radians(90.0 - theta_d)) * np.sin(np.radians(phi_d))
    z = np.cos(np.radians(90.0 - theta_d))
    return array([x, y, z])

In [15]:
def spheric_cartesian_elevation(phi_d, theta_d):
    x = np.cos(np.radians(theta_d)) * np.cos(np.radians(phi_d))
    y = np.cos(np.radians(theta_d)) * np.sin(np.radians(phi_d))
    z = np.sin(np.radians(theta_d))
    return array([x, y, z])

### Length of Vector

In [16]:
def vectorlength(vector):
    return np.linalg.norm(vector)

### Calculated Angular Error

In [17]:
def calculate_angular_error(deg_e_phi, deg_e_theta):
    return np.degrees(np.arccos(np.cos(np.radians(deg_e_phi)) * np.cos(np.radians(deg_e_theta))))

### Skalarprodukt

In [18]:
def myDot(a, b):
    dot = 0;
    it = np.nditer(a, flags=['f_index'])
    for x in it:
        dot = dot + (x * b[it.index])
        
    return dot

### Dot Angular Error

In [19]:
def dot_angular_error_elevation(predicted_deg_vector, true_deg_vector):    
    c_predicted = spheric_cartesian_elevation(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_elevation(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

def dot_angular_error_polar(predicted_deg_vector, true_deg_vector):
    c_predicted = spheric_cartesian_polar(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_polar(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

### Automatische Optimizer Generierung aus String

In [20]:
def make_optimizer(optimizer):
    # [Adam, Nadam, Adagrad, RMSprop]
    if optimizer == "<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>":
        return Adam
    else:
        print('ERROR::: Unspecified Optimizer')

### Convert String into Reduction Metric Function

In [21]:
def get_reduction_metric(metric):
    
    if metric == 'mean_absolut_error':
        return [mean_absolut_error]
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

## Normierte sphärische Koordinaten

### Evaluate

In [22]:
def evaluate(eval_df, eval_predictions, save_dir = None, save_to_file = False):
    num_predictions = eval_df.shape[0]
    
    df_result = pd.DataFrame({
        'Filename': eval_df['Filename'][0:num_predictions],
        'Elevation_true': eval_df['Elevation'][0:num_predictions],
        'Elevation_pred': eval_predictions[:, 0],
        'Elevation_err': None,
        'Azimuth_true': eval_df['Azimuth'][0:num_predictions],
        'Azimuth_pred': eval_predictions[:, 1],
        'Azimuth_err': None,
        'Angular_MAE': None,
        'dot_angular_err_elevation': None,
        'dot_angular_err_polar': None,
    })

    angular_calculated = ''

    #df_result['Elevation_pred'] = eval_predictions[:, 0] #todo nach oben? 
    #df_result['Azimuth_pred'] = eval_predictions[:, 1]

    for index, row in df_result.iterrows():
        #print('=======================================================================')
        predicted_azimuth = row['Azimuth_pred']
        predicted_elevation = row['Elevation_pred']

        true_azimuth = row['Azimuth_true']
        true_elevation = row['Elevation_true']

        # azimuth and elevation error
        error_elevation = abs((predicted_elevation - true_elevation))
        df_result.at[index, 'Elevation_err'] = error_elevation

        error_azimuth = min(abs(predicted_azimuth - true_azimuth), abs(360 - abs(predicted_azimuth - true_azimuth)))
        df_result.at[index, 'Azimuth_err'] = error_azimuth

        # dot angular error
        predicted_direction = array([predicted_azimuth, predicted_elevation])
        true_direction = array([true_azimuth, true_elevation])

        error_direction_elevation = dot_angular_error_elevation(predicted_direction, true_direction)
        df_result.at[index, 'dot_angular_err_elevation'] = error_direction_elevation
        error_direction_polar = dot_angular_error_polar(predicted_direction, true_direction)
        df_result.at[index, 'dot_angular_err_polar'] = error_direction_polar


        # Angular Mean Absolute Error
        y_pred = K.constant(predicted_direction)
        y_true = K.constant (true_direction)
        mae = mean_absolut_error(y_true, y_pred)
        df_result.at[index, 'Angular_MAE'] = mae
        #print('=======================================================================')

    error_elevation_avg = np.mean(df_result['Elevation_err'], axis = 0)
    error_azimuth_avg = np.mean(df_result['Azimuth_err'], axis = 0)
    
    
    # Box'n Whiskers Diagram Data - Angular Error
    e_angular_median = np.quantile(df_result['dot_angular_err_elevation'], 0.5)
    e_angular_lower_q = np.quantile(df_result['dot_angular_err_elevation'], 0.25)
    e_angular_upper_q = np.quantile(df_result['dot_angular_err_elevation'], 0.75)
    e_angular_min = np.amin(df_result['dot_angular_err_elevation'])
    e_angular_max = np.amax(df_result['dot_angular_err_elevation'])
    
    max_angular_error = df_result.loc[df_result['dot_angular_err_elevation'] == e_angular_max]
    max_angular_error_file = max_angular_error.iloc[0]['Filename']
    max_angular_error_theta_true = max_angular_error.iloc[0]['Elevation_true']
    max_angular_error_phi_true = max_angular_error.iloc[0]['Azimuth_true']
    max_angular_error_theta_pred = max_angular_error.iloc[0]['Elevation_pred']
    max_angular_error_phi_pred = max_angular_error.iloc[0]['Azimuth_pred']
    
    
    # Box'n Whiskers Diagram Data - Mean Absolute Error
    e_mae_median = np.quantile(df_result['Angular_MAE'], 0.5)
    e_mae_lower_q = np.quantile(df_result['Angular_MAE'], 0.25)
    e_mae_upper_q = np.quantile(df_result['Angular_MAE'], 0.75)
    e_mae_min = np.amin(df_result['Angular_MAE'])
    e_mae_max = np.amax(df_result['Angular_MAE'])
    
    max_mae_error = df_result.loc[df_result['Angular_MAE'] == e_mae_max]
    max_mae_error_file = max_mae_error.iloc[0]['Filename']
    max_mae_error_theta_true = max_mae_error.iloc[0]['Elevation_true']
    max_mae_error_phi_true = max_mae_error.iloc[0]['Azimuth_true']
    max_mae_error_theta_pred = max_mae_error.iloc[0]['Elevation_pred']
    max_mae_error_phi_pred = max_mae_error.iloc[0]['Azimuth_pred']
    
    # Avg Angular Error
    error_dot_angular_elevation_avg = np.mean(df_result['dot_angular_err_elevation'], axis = 0)
    angular_variance = np.var(df_result['dot_angular_err_elevation'], axis = 0)
    avg_angular_lower_mean = np.mean(df_result['dot_angular_err_elevation'][df_result['dot_angular_err_elevation'] < error_dot_angular_elevation_avg])
    avg_angular_upper_mean = np.mean(df_result['dot_angular_err_elevation'][df_result['dot_angular_err_elevation'] > error_dot_angular_elevation_avg])
    
    # Avg MAE Error
    avg_mae_mean = np.mean(df_result['Angular_MAE'])
    avg_mae_lower_mean = np.mean(df_result['Angular_MAE'][df_result['Angular_MAE'] < avg_mae_mean])
    avg_mae_upper_mean = np.mean(df_result['Angular_MAE'][df_result['Angular_MAE'] > avg_mae_mean])
    

    print('Durchschnnittlicher Fehler Elevation{}: {:.1f}'.format(angular_calculated, error_elevation_avg))
    print('Durchschnnittlicher Fehler Azimut{}: {:.1f}'.format(angular_calculated, error_azimuth_avg))
    print()
    print('Durchschnnittlicher Winkelfehler{} (Elevation): {:.1f}'.format(angular_calculated, error_dot_angular_elevation_avg))
    print('Varianz Winkelfehler{}: {:.1f}'.format(angular_calculated, angular_variance))
    print()
    print('Box-Whisker Angular: max - {:.1f}, upper - {:.1f}, median - {:.1f}, lower - {:.1f}, min - {:.1f}'.format(e_angular_max, e_angular_upper_q, e_angular_median, e_angular_lower_q, e_angular_min))
    print('Maxium Angular Error on: {}, True ({:.1f}, {:.1f}), Pred ({:.1f}, {:.1f})'.format(max_angular_error_file, max_angular_error_phi_true, max_angular_error_theta_true, max_angular_error_phi_pred, max_angular_error_theta_pred))
    print('Box-Whisker MAE: max - {:.1f}, upper - {:.1f}, median - {:.1f}, lower - {:.1f}, min - {:.1f}'.format(e_mae_max, e_mae_upper_q, e_mae_median, e_mae_lower_q, e_mae_min))
    print('Maxium MAE Error on: {}, True ({:.1f}, {:.1f}), Pred ({:.1f}, {:.1f})'.format(max_mae_error_file, max_mae_error_phi_true, max_mae_error_theta_true, max_mae_error_phi_pred, max_mae_error_theta_pred))
    print()
    print('Avg Angular Range: lower - {:.1f}, avg - {:.1f}, upper - {:.1f}'.format(avg_angular_lower_mean, error_dot_angular_elevation_avg, avg_angular_upper_mean))
    print('Avg MAE Range: lower - {:.1f}, avg - {:.1f}, upper - {:.1f}'.format(avg_mae_lower_mean, avg_mae_mean, avg_mae_upper_mean))
    
        
    df_avg = pd.DataFrame({
        'Avg_Elevation_Err': [error_elevation_avg],
        'Avg_Azimuth_Err': [error_azimuth_avg],
        'Avg_Angular_Err': [error_dot_angular_elevation_avg],
        'Angular_Variance': [angular_variance],
        'Avg_Inference_Time': [image_time],
        'box_angular_medium': [e_angular_median],
        'box_angular_lower': [e_angular_lower_q],
        'box_angular_upper': [e_angular_upper_q],
        'box_angular_min': [e_angular_min],
        'box_angular_max': [e_angular_max],
        'box_mae_medium': [e_mae_median],
        'box_mae_lower': [e_mae_lower_q],
        'box_mae_upper': [e_mae_upper_q],
        'box_mae_min': [e_mae_min],
        'box_mae_max': [e_mae_max],
        'avg_angular_lower_mean': [avg_angular_lower_mean],
        'avg_angular_upper_mean': [avg_angular_upper_mean],
        'avg_mae_mean': [avg_mae_mean],
        'avg_mae_lower_mean': [avg_mae_lower_mean],
        'avg_mae_lower_mean': [avg_mae_upper_mean],
        'max_angular_error_file': [max_angular_error_file],
        'max_angular_error_theta_true': [max_angular_error_theta_true],
        'max_angular_error_phi_true': [max_angular_error_phi_true],
        'max_angular_error_theta_pred': [max_angular_error_theta_pred],
        'max_angular_error_phi_pred': [max_angular_error_phi_pred],
        'max_mae_error_file': [max_mae_error_file],
        'max_mae_error_theta_true': [max_mae_error_theta_true],
        'max_mae_error_phi_true': [max_mae_error_phi_true],
        'max_mae_error_theta_pred': [max_mae_error_theta_pred],
        'max_mae_error_phi_pred': [max_mae_error_phi_pred]
    })

    if(not os.path.exists(Global.evaluation_path)):
        os.makedirs(Global.evaluation_path)
    df_result.to_csv(Global.evaluation_path + 'Model-Testset-{}_Prediction_Results.csv'.format(Global.dataset), index=False)
    df_avg.to_csv(Global.evaluation_path + 'Model-Testset-{}_Average_Results.csv'.format(Global.dataset), index=False)
    
    if save_to_file:
        df_result.to_csv(save_dir + 'Prognosen.csv')#.format('%.2f'%error_elevation_avg, '%.2f'%error_azimuth_avg), index=False)
        #df_result.to_csv(save_dir + 'Prognosen_ErrE_{}_ErrA_{}.csv'.format('%.2f'%error_elevation_avg, '%.2f'%error_azimuth_avg), index=False)
    
    return df_result, df_avg

# Test the Model <a name = "setup"></a>

## Hilfsdaten

In [23]:
mse_results = []

## Evaluation Setup
<p><a href = #Top>Top</a> 
<p><a href = #store>Save File</a>

#### Required format of parameters parameter for _model_predict_ (...)

In [24]:
p = {
    'model_to_load':[],
    'optimizer':[],
    'learning_rate':[],
    'first_neuron':[],
    'dropout_rate':[],
    'activation':[],
    'leaky_alpha':[],
    'hidden_layers':[],
}

### Struct for global parameter

In [25]:
@dataclass
class global_parameter:
    #========================================================
    # just change this, everything else will automaticlly adjusted
    
    net_architecture = 'RESNET' # 'ALEX' vs 'VGG16' vs 'RESNET' vs 'preVGG16'
    image_channels: str = 'rgb' # 'rgb' vs 'rgba'
    mode: str = 'SYNTH' # 'SYNTH' vs 'REAL' 'MIXED'
    #======================================================== 
    
    loss_function: str = 'mean_squared_error'
    reduction_metric: str = 'mean_absolut_error'
    monitor_value: str = 'val_mean_absolut_error'
            
    dataset: str = '201129_2031' #'201019_2253_final' vs 201129_2031'
    device: str = 'RTX_2080_Ti'
    data_augmentation: bool = True
    image_dir: str = '..\\..\\data_generation\\dataset\\{}\\'.format(dataset)
    
    # Default values for rgb -> later ajusted when rgba
    num_image_channels: int = 3
    csv_file_name: str = 'labels_ks_RGB.csv'
    csv_file: str = image_dir + csv_file_name
    model_dir: str = '..\\output\\{}_{}_{}\\'.format(net_architecture, dataset, image_channels)
    #results: str = '\\..\\{}_{}_Results.csv'.format(net_architecture, dataset)
    #results_man: str = '\\..\\ALEX_201019_2253_final_Results_manual.csv'
    
    run: str = datetime.now().strftime('%Y%m%d_%H%M')
    print(run)
    evaluation_path = model_dir + 'Evaluation_{}_{}\\'.format(mode, run)
    talos_results: str = 'Talos_Results_Fine_Idx_{}_{}.csv'.format(net_architecture, image_channels) 
    model_path = model_dir + 'CNN_{}_Model_and_Weights_{}.hdf5'.format(net_architecture, image_channels)
    
Global = global_parameter
print(Global.csv_file)

if(Global.image_channels == 'rgba'):
    Global.num_image_channels = 4
    Global.csv_file_name = 'labels_ks_RGBD.csv'
    Global.csv_file = Global.image_dir + Global.csv_file_name
    model_dir = '..\\output\\{}_{}_{}\\'.format(Global.net_architecture, Global.dataset, Global.image_channels)
    #results: str = '\\..\\{}_{}_Results.csv'.format(Global.net_architecture, Global.dataset)
    talos_results = 'Talos_Results_Fine_Idx_{}_{}.csv'.format(Global.net_architecture, Global.image_channels)
    model_path = Global.model_dir + 'CNN_{}_Model_and_Weights_{}.hdf5'.format(Global.net_architecture, Global.image_channels)
    print(Global.csv_file)
    
if(Global.net_architecture == 'preVGG16'):
    n = 210 # 198 vs 210
    Global.model_dir = '..\\output\\SYNTH_Regression_MSE\\201129_2031_Angular_Top_1_Custom-MAE\\Synth_TD\\'
    Global.talos_results = 'Talos_Results_Fine_Idx{}.csv'.format(n)
    Global.model_path = Global.model_dir + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(n, 80000)

if(Global.mode == 'REAL'):
    Global.image_dir = '..\\dataset_mm\\2020-05-28\\'
    Global.csv_file = Global.image_dir + 'images_real.csv'

20201208_1145
..\..\data_generation\dataset\201129_2031\labels_ks_RGB.csv


### Test to see if RGBD

In [26]:
from PIL import Image 
df2 = pd.read_csv(Global.csv_file)
print(df2.head(5))
img_nm = df2.iloc[0]['Filename']

img = Image.open(Global.image_dir + img_nm)
img.show()

for i in range(224):
    for j in range(224):
        colors = img.getpixel((i,j))
        print(colors)

                                 Filename  Azimuth  Elevation
0   buddha/rgb/buddha00000000-0-5-0-5.png        0          5
1  buddha/rgb/buddha00000001-0-5-0-10.png        0         10
2  buddha/rgb/buddha00000002-0-5-0-15.png        0         15
3  buddha/rgb/buddha00000003-0-5-0-20.png        0         20
4  buddha/rgb/buddha00000004-0-5-0-25.png        0         25
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49

...

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 2

(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 49)
(49, 49, 4

#####  Trainiert mit: Synthetische Daten

In [27]:
network_file = Global.model_dir + Global.talos_results
dataframe = pd.read_csv(network_file)

dataframe.head()

,round_epochs,loss,mean_absolut_error,val_loss,val_mean_absolut_error,lr,activation,batch_size,dropout,epochs,first_neuron,hidden_layers,leaky_alpha,lr.1,optimizer,samples,Base
0,173,334.156501,5.267704,390.986777,5.162872,0.00001,relu,16,0.25,400,1024,0,0.1,1,<class 'tensorflow.python.keras.optimizer_v2.a...,100000,rgb


In [28]:
current_params = load_params(dataframe, Global.model_path)
print(current_params)

{'model_to_load': '..\\output\\RESNET_201129_2031_rgb\\CNN_RESNET_Model_and_Weights_rgb.hdf5', 'optimizer': <class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>, 'learning_rate': 1e-05, 'first_neuron': 1024, 'dropout_rate': 0.25, 'activation': 'relu', 'leaky_alpha': 0.1, 'hidden_layers': 0}


In [30]:
with tf.device('/device:GPU:1'):
    predictions, df_test, duration, image_time = model_predict(current_params)

Found 10000 validated image filenames.
Net: RESNET
Image Channels: rgb
test_generator: <keras_preprocessing.image.dataframe_iterator.DataFrameIterator object at 0x0000019F77D48248>
test_generator.n: 10000
test_generator.batch_size: 1
steps: 10000


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv1_conv/Conv2D (defined at <ipython-input-10-3aad9fcb0cf4>:50) ]] [Op:__inference_distributed_function_15978]

Function call stack:
distributed_function


In [ ]:
with tf.device('/device:GPU:1'):
    result_df, avg_df = evaluate(df_test, predictions,  save_dir = Global.evaluation_path, save_to_file = True)
    mse_results.append([network_file, result_df, avg_df])

# Evaluation <a name = "store">
<p></a><a href = #Top>Top</a>
<p><a href = #setup>Setup</a>

In [ ]:
eval_dir = '..\\output\\Regression\\{}__{}__{]Graphical_Evaluation\\'.format(Global.net_architecture, Global.image_channels, Global.mode)

if(not os.path.exists(eval_dir)):
    os.makedirs(eval_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(eval_dir))

In [ ]:
import pickle

with open(eval_dir + '{}_{}_{}_Results.pickle'.format(Global.net_architecture, Global.image_channels, Global.mode), "wb") as fp:   #Pickling
    pickle.dump(mse_results, fp) # uncomment if you are REALLY sure to overwrite the file

with open(eval_dir + '{}_{}_{}_Results.pickle'.format(Global.net_architecture, Global.image_channels, Global.mode), "rb") as fp:   # Unpickling
    b = pickle.load(fp)